In [2]:
import sys

sys.path.append("../berries")


In [3]:
from datasets import load_dataset

mnist = load_dataset("mnist").with_format("jax")
mnistData = mnist['train']
X_img = mnistData['image']
y = mnistData['label']
X_img_test = mnist["test"]["image"]
n_test_samples = X_img_test.shape[0]
X_test = X_img_test.reshape((n_test_samples, -1))
y_test = mnist["test"]["label"]
n_samples, _, _ = X_img.shape
X = X_img.reshape((n_samples, -1))
n_samples, d_x = X.shape
d_y = len(set(y.tolist()))

/home/owner/Projects/picojam/pico/.venv/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for mnist contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mnist
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [7]:
d_h_layer = 128
d_h = 64
n_layers = 1
d_in = d_x
# d_out = d_y
d_out = d_x


In [15]:
import optax
from jax.numpy import mean
from jax import grad, jit
from jax.tree_util import tree_map
from pf import F, _
# import nn
# import importlib
# importlib.reload(nn)
import random_utils
from nn import mglu_config, mglu, init_weights, fmt_weights

seed = 0
key_gen = random_utils.infinite_safe_keys(seed)

def accuracy(logits, y):
    return (logits.argmax(-1) == y).mean()


mglu_b = F(mglu).vmap((0, None, None), 0)

def sce_loss(to_logits, x, y, W):
    return optax.softmax_cross_entropy_with_integer_labels(to_logits(x, W), y)

def l2_loss(to_logits, x, W):
    return ((to_logits(x, W) - x) ** 2).mean()



loss_b_all = F(l2_loss).f(mglu,  _, _).vmap((0, None), 0)
loss_d = loss_b_all.f(X, _)
loss_b_d = loss_d >> mean

lr = 0.001
mask_fn = lambda p: tree_map(lambda x: not isinstance(x, int), p)
# opt = optax.multi_transform({"sgd": optax.rmsprop(lr), "zero": optax.set_to_zero()}, mask_fn)
opt = optax.masked(optax.rmsprop(lr), mask_fn)


method = {"type": "zer0", "std": 0.001}
# method = {"type": "normal", "std": 0.01}
W = init_weights(next(key_gen), mglu_config(d_in, d_h_layer, d_out, d_h, n_layers, method))
loss0 = loss_b_d(W)
print(loss0)
print(fmt_weights(W)[0])
state = opt.init(W)

@jit
def update(W, opt_state):
    grads = grad(loss_b_d)(W)
    updates, opt_state = opt.update(grads, opt_state)
    new_W = optax.apply_updates(W, updates)
    return new_W, opt_state

7283.9497
tuple:
    sglu:
        wv:
            array shape: (784, 64)
        wu:
            array shape: (784, 64)
        wo:
            array shape: (64, 128)
        total params: 108544
    rmsn:
        d:
            128.0
        total params: 1
    total params: 108545
tuple:
    sglu:
        wv:
            array shape: (128, 64)
        wu:
            array shape: (128, 64)
        wo:
            array shape: (64, 784)
        total params: 66560
    rmsn:
        d:
            784.0
        total params: 1
    total params: 66561
total params: 175106



In [16]:
def get_accuracy(x, y, W):
    return accuracy(mglu(x, W), y)

get_accuracy_b = F(get_accuracy).vmap(in_axes=(0, 0, None), out_axes=0)
get_accuracy_b_d = get_accuracy_b.f(X, y, _) >> mean
get_accuracy_b_t = get_accuracy_b.f(X_test, y_test, _) >> mean

for i in range(1000):
    W, state = update(W, state)
    if i % 50 == 0:
        print(get_accuracy_b_d(W), get_accuracy_b_t(W), loss_b_d(W))
print(loss_d(W))

0.00075 0.0008 7244.5195
0.0 0.0 7174.575
1.6666667e-05 0.0 7158.53
0.0 0.0 7151.519
0.0 0.0 7147.421
1.6666667e-05 0.0 7144.5034
1.6666667e-05 0.0 7142.6255
3.3333334e-05 0.0 7140.2344
0.0 0.0 7138.8994
0.0 0.0 7137.719
0.0 0.0 7136.717
0.0 0.0 7135.7485
0.0 0.0 7135.24
0.0 0.0 7134.5835
0.0 0.0 7133.6973
0.0 0.0 7133.7935
0.0 0.0 7132.9688
0.0 0.0 7132.452
0.0 0.0 7132.4087
0.0 0.0 7131.6704
[7421.765  8438.289  4808.6577 ... 5775.553  4974.0103 5157.831 ]
